In [2]:
!pip install -q hopsworks[python]

In [3]:
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks


import warnings
warnings.filterwarnings("ignore")

# 1. Read your Hopsworks API Key from a file or environment variable
#    For example, if stored in 'hopsworks-api-key.txt'
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

project = hopsworks.login()
fs = project.get_feature_store()

# Retrieve feature groups for Clash Royale
player_stats_fg = fs.get_feature_group(
    name='clash_royale_onehotencoding',
    version=5,
)
game_events_fg = fs.get_feature_group(
    name='clash_royale_dataset_onehotencoding',
    version=5,
)

df = player_stats_fg.read()

2025-01-08 12:21:00,118 INFO: Initializing external client
2025-01-08 12:21:00,132 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 12:21:02,917 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.60s) 


In [4]:
dfstats = player_stats_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.53s) 


In [5]:
dfdata = game_events_fg.read()
dfdata

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (124.03s) 


,player_name,player_name2,deck1,deck2,result
0,#PRG9UCG8C,#2URQP2CJ,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",1
1,#QY8Q9LGY,#GG0QRG8Y,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,#URQJURR0,#9U8U9PV92,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,#2V8G9RPQ8,#LYPV9GPC,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,#PUVYRQRC,#2JC9PQ2L,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...
399842,#22GC8P92,#22LJP8CYQ,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
399843,#VCL2QYJG,#2Y0CJU9RQ,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
399844,#PRQUQL9YY,#Y28JJ9G0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
399845,#2YQ9RPJ20,#PCL8GG2G,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [6]:
dfdata

,player_name,player_name2,deck1,deck2,result
0,#PRG9UCG8C,#2URQP2CJ,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",1
1,#QY8Q9LGY,#GG0QRG8Y,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,#URQJURR0,#9U8U9PV92,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,#2V8G9RPQ8,#LYPV9GPC,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,#PUVYRQRC,#2JC9PQ2L,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...
399842,#22GC8P92,#22LJP8CYQ,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
399843,#VCL2QYJG,#2Y0CJU9RQ,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
399844,#PRQUQL9YY,#Y28JJ9G0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
399845,#2YQ9RPJ20,#PCL8GG2G,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [34]:
df_1 = dfdata.iloc[:10000,:]
df_test = dfdata.iloc[20000:21000, :]

In [35]:
df_test['result']

20000    1
20001    1
20002    1
20003    1
20004    1
        ..
20995    1
20996    1
20997    1
20998    1
20999    1
Name: result, Length: 1000, dtype: int64

In [36]:
# prompt: what are the different values for the column result

import pandas as pd

# Assuming dfdata is your DataFrame
# Replace with your actual DataFrame if it's named differently

# Check for unique values in the 'result' column
unique_results = dfdata['result'].unique()

# Print the unique values
unique_results

array([1], dtype=int64)

In [37]:
import pandas as pd

# Assuming the DataFrame is named df_1
# Create binary columns for deck1
deck1_columns = [f"deck1_{i}" for i in range(181)]
deck1_df = pd.DataFrame(df_1['deck1'].tolist(), columns=deck1_columns)

# Create binary columns for deck2
deck2_columns = [f"deck2_{i}" for i in range(181)]
deck2_df = pd.DataFrame(df_1['deck2'].tolist(), columns=deck2_columns)

# Combine all columns with the result column
result_column = df_1['result']
final_df = pd.concat([deck1_df, deck2_df, result_column], axis=1)

# Display or save the final DataFrame
final_df.head()  # To preview the DataFrame
# You can save the DataFrame to a CSV file if needed:


,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [38]:
# Count non-zero elements in each row for deck1 and deck2
deck1_non_zero_count = (deck1_df != 0).sum(axis=1)
deck2_non_zero_count = (deck2_df != 0).sum(axis=1)

print(deck1_non_zero_count)

0       8
1       8
2       8
3       8
4       7
       ..
9995    7
9996    8
9997    8
9998    8
9999    8
Length: 10000, dtype: int64


In [39]:
'''
DOBBIAMO CREARE ANCHE I DATI OPPOSTI!
'''

'\nDOBBIAMO CREARE ANCHE I DATI OPPOSTI!\n'

In [40]:
# Create a new DataFrame with deck2 first, then deck1, and result column set to 0
result_column_zero = pd.Series([0] * len(df_1), name="result")
reversed_df = pd.concat([deck2_df, deck1_df, result_column_zero], axis=1)
reversed_df

,deck2_0,deck2_1,deck2_2,deck2_3,deck2_4,deck2_5,deck2_6,deck2_7,deck2_8,deck2_9,...,deck1_172,deck1_173,deck1_174,deck1_175,deck1_176,deck1_177,deck1_178,deck1_179,deck1_180,result
0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df_complete = pd.concat([final_df, reversed_df], ignore_index=True)
df_complete

,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df_shuffled = df_complete.sample(frac=1, random_state=42).reset_index(drop=True)
final_df = df_shuffled.copy()
final_df

,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19998,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [43]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Prepare data
X = final_df.iloc[:, :-1].values  # Features: all columns except the result
y = final_df.iloc[:, -1].values  # Target: the result column

In [44]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='tanh')  # Output layer with tanh activation for -1 to 1 range
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4588 - loss: 0.3101 - val_accuracy: 0.4216 - val_loss: 0.2724
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5133 - loss: 0.2542 - val_accuracy: 0.3750 - val_loss: 0.2827
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5450 - loss: 0.2476 - val_accuracy: 0.3269 - val_loss: 0.2866
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5600 - loss: 0.2437 - val_accuracy: 0.3025 - val_loss: 0.2949
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5716 - loss: 0.2407 - val_accuracy: 0.2894 - val_loss: 0.3025
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5731 - loss: 0.2389 - val_accuracy: 0.2713 - val_loss: 0.3114
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5978 - loss: 0.2345 - val_accuracy: 0.2603 - val_loss: 0.3209
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6019 - loss: 0.2328 - val_accuracy: 0.

In [45]:
y = (y == 1).astype(int)  # Map -1 to 0 and keep 1 as 1
y

array([0, 1, 1, ..., 1, 1, 0])

In [46]:
print("Unique classes in y after transformation:", set(y))

Unique classes in y after transformation: {0, 1}


In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [48]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the model
xgb_model.fit(X_train, y_train)

# Predict outcomes for the test set
y_pred = xgb_model.predict(X_test)

# Invert predictions if necessary
y_pred_corrected = 1 - y_pred  # Flip the predictions

# Evaluate the model with corrected predictions
xgb_accuracy_corrected = accuracy_score(y_test, y_pred_corrected)
xgb_report_corrected = classification_report(y_test, y_pred_corrected)

# Print results
print(f"XGBoost Test Accuracy (Corrected): {xgb_accuracy_corrected:.4f}")
print("Classification Report (Corrected Predictions):\n", xgb_report_corrected)


XGBoost Test Accuracy (Corrected): 0.8760
Classification Report (Corrected Predictions):
               precision    recall  f1-score   support

           0       0.88      0.87      0.88      2000
           1       0.87      0.88      0.88      2000

    accuracy                           0.88      4000
   macro avg       0.88      0.88      0.88      4000
weighted avg       0.88      0.88      0.88      4000



In [22]:
df_test

,player_name,player_name2,deck1,deck2,result
10000,#8L0RJ9JL,#QLPUJGJ,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
10001,#J0VPRY2L,#29PPJL8JL,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",1
10002,#Q8G29UPGL,#8RYCUQVG,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
10003,#PQU2Q0JRV,#UL22C0VP,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",1
10004,#GRP9JVV2C,#2J80U0L8U,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...
10995,#8J88YLJU0,#C22QGRLL,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
10996,#PQ888JR28,#9YLJVC9LL,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1
10997,#JGGVYURV,#CVQ9GPPV,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ...",1
10998,#90UJPLCQQ,#8CR9G8PCV,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [49]:
df_test['result']

20000    1
20001    1
20002    1
20003    1
20004    1
        ..
20995    1
20996    1
20997    1
20998    1
20999    1
Name: result, Length: 1000, dtype: int64

In [24]:
# Creare le colonne binarie per deck1
deck1_test_df = pd.DataFrame(df_test['deck1'].tolist(), columns=deck1_columns).reset_index(drop=True)

# Creare le colonne binarie per deck2
deck2_test_df = pd.DataFrame(df_test['deck2'].tolist(), columns=deck2_columns).reset_index(drop=True)

# Combinare tutte le colonne con la colonna result
result_column_test = df_test['result'].reset_index(drop=True)
final_test_df = pd.concat([deck1_test_df, deck2_test_df, result_column_test], axis=1).reset_index(drop=True)

# Contare gli elementi non-zero per ogni riga in deck1 e deck2 (opzionale, se richiesto)
deck1_test_non_zero_count = (deck1_test_df != 0).sum(axis=1)
deck2_test_non_zero_count = (deck2_test_df != 0).sum(axis=1)
final_test_df

,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [25]:
X_test = final_test_df.iloc[:, :-1].values
y_test = final_test_df.iloc[:, -1].values

In [26]:
# Predict outcomes for the test set
y_pred = xgb_model.predict(X_test)

# Invert predictions if necessary
y_pred_corrected = 1 - y_pred  # Flip the predictions

# Evaluate the model with corrected predictions
xgb_accuracy_corrected = accuracy_score(y_test, y_pred)
xgb_report_corrected = classification_report(y_test, y_pred)

# Print results
print(f"XGBoost Test Accuracy (Corrected): {xgb_accuracy_corrected:.4f}")
print("Classification Report (Corrected Predictions):\n", xgb_report_corrected)

XGBoost Test Accuracy (Corrected): 0.4910
Classification Report (Corrected Predictions):
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.49      0.66      1000

    accuracy                           0.49      1000
   macro avg       0.50      0.25      0.33      1000
weighted avg       1.00      0.49      0.66      1000



In [27]:
# Predicting target values on the test set

# Calculating Mean Squared Error (MSE) using sklearn
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# Calculating R squared using sklearn
r2 = r2_score(y_test, y_pred)
print("R squared:", r2)

MSE: 0.509
R squared: 0.0


In [50]:
# Print 10 examples of y_test and y_pred_corrected
print("\nExamples of y_test and Corrected Predictions:")
for i in range(100):
    print(f"{y_test[i]} {y_pred[i]}")


Examples of y_test and Corrected Predictions:
1 0
0 0
0 0
1 0
0 1
0 1
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 1
1 0
0 1
1 0
0 1
1 1
0 0
0 0
0 1
1 0
1 0
0 1
0 1
1 0
1 0
1 1
1 1
1 0
1 0
0 1
1 0
1 0
0 1
1 0
0 1
1 0
1 0
0 0
0 1
1 1
0 1
0 1
0 1
1 0
1 0
1 0
1 0
0 1
0 1
0 1
0 1
1 0
1 0
0 1
1 0
0 1
1 0
1 0
1 0
0 0
0 1
1 1
0 1
0 1
0 1
0 1
1 0
1 0
1 0
1 0
1 0
0 1
0 1
0 1
1 0
0 1
1 0
1 0
0 1
0 1
0 1
1 0
1 0
0 1
1 1
1 0
1 0
0 1
1 0
0 1
0 1
0 1
0 0
1 0


In [51]:
# Creating a directory for the model artifacts if it doesn't exist
model_dir = "clash_royale_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [30]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Creating input and output schemas using the 'Schema' class for features (X) and target variable (y)
input_schema = Schema(X_train)
output_schema = Schema(y_train)

# Creating a model schema using 'ModelSchema' with the input and output schemas
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

# Converting the model schema to a dictionary representation
schema_dict = model_schema.to_dict()

In [31]:
xgb_model.save_model(model_dir + "/model.json")

In [32]:
res_dict = { 
        "MSE": str(mse),
        "R squared": str(r2),
    }

In [33]:
mr = project.get_model_registry()

# Creating a Python model in the model registry named 'air_quality_xgboost_model'

# Convert X_test to a DataFrame to use the sample method
X_test_df = pd.DataFrame(X_test)

aq_model = mr.python.create_model(
    name="clash_royale_xgboost_model", 
    metrics= res_dict,
    model_schema=model_schema,
    input_example=X_test_df.sample().values, 
    description="Win Probability rate",
)

# Saving the model artifacts to the 'air_quality_model' directory in the model registry
aq_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/510677 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1088 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/226 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1175700/models/clash_royale_xgboost_model/2


Model(name: 'clash_royale_xgboost_model', version: 2)